In [33]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm as tqdm
import numpy as np
from sklearn.decomposition import PCA
from sklearn import linear_model
import sklearn
from sklearn.metrics import mean_squared_error

In [34]:
data = pd.read_csv("data.csv")
data = data.dropna()
data = data.drop(['position','name','age','club','nationality'], axis = 1)
for i in range(0,data.shape[0]):
    data['fpl_sel'].iloc[i] = float(str(data['fpl_sel'].iloc[i]).rstrip('%'))
data['fpl_sel'] = data['fpl_sel'].astype('float64')

# outliers removal
# print(data['page_views'].quantile(0.90))
# data["page_views"] = np.where(data["page_views"] >1713.4, 1713.4,data['page_views'])
# print(data['fpl_sel'].quantile(0.90))
# data["fpl_sel"] = np.where(data["fpl_sel"] >10.4, 10.4,data['fpl_sel'])

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [35]:
#splitting the data
X = data[data.columns[~data.columns.isin(['market_value'])]]
y = data['market_value']

from sklearn.model_selection import train_test_split
np.random.seed(37) # Set seed
x_train, x_test = train_test_split(X, test_size = 0.15, random_state = 40 )

## dont using iloc as we have projected from data
y_train = y.loc[x_train.index.values] 
y_test = y.loc[x_test.index.values]
x_train = X.loc[x_train.index.values, :]
x_test = X.loc[x_test.index.values, :]

In [36]:
concat_train_data = pd.concat([x_train,y_train],axis=1)

#scaling
from sklearn.preprocessing import StandardScaler
scaler =StandardScaler()
scaled_data = scaler.fit_transform(x_train)
#print(scaler.transform(concat_train_data))
scaled_data.shape

(391, 11)

In [37]:
pca = PCA(n_components=11)
pca.fit(scaled_data)
print(pca.explained_variance_)
print(np.cumsum(pca.explained_variance_ratio_*100))

#taking all the components which explains 100% varience

[3.13518941 1.42908526 1.32032697 1.07965441 0.97622187 0.92121488
 0.69508833 0.63607002 0.39216154 0.29188226 0.15131017]
[ 28.42882752  41.38728487  53.35955923  63.14949695  72.00154363
  80.3548057   86.65762941  92.42529528  95.98128231  98.627971
 100.        ]


In [38]:
pca = PCA(n_components=11)
pca.fit(scaled_data)

PCA(n_components=11)

In [44]:
X = pca.transform(scaled_data)
y = y_train

#linear_Model

from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, y)

In [43]:
#scaling and pca on the test data
scaled_data_test = scaler.fit_transform(x_test)
test_data = pca.transform(scaled_data_test)
pred = reg.predict(test_data)
mean_squared_error(y_test,pred)

28.359678263552663

In [41]:
#R2 value on test data
sklearn.metrics.r2_score(y_test, pred, sample_weight=None, multioutput='uniform_average')

0.8102008920434557

In [42]:
# MSE of test data
mean_squared_error(y_test,pred)

28.359678263552663

In [49]:
#ridge model
from sklearn import linear_model

clf = linear_model.Ridge(alpha=0.1,max_iter=800,tol = 0.000001)
clf.fit(X,y)
from sklearn.metrics import mean_squared_error
pred_lasso = clf.predict(test_data)
mean_squared_error(y_test,pred_lasso)

28.36213748092912

In [50]:
sklearn.metrics.r2_score(y_test, pred_lasso, sample_weight=None, multioutput='uniform_average')

0.8101844335610997

In [53]:
#lasso model
from sklearn import linear_model

clf = linear_model.Lasso(alpha=0.1,max_iter=800,tol = 0.000001)
clf.fit(X,y)
from sklearn.metrics import mean_squared_error
pred_lasso = clf.predict(test_data)
mean_squared_error(y_test,pred_lasso)

28.697324630478736

In [48]:
clf.n_iter_

2

In [25]:
import sklearn
sklearn.metrics.r2_score(y_test, pred_lasso, sample_weight=None, multioutput='uniform_average')

0.8102005394672662